光流的生成

In [26]:
import numpy as np
import torch


def gen_flow_circle(center,height,width):
    x0,y0=center
    
    if x0>=height or y0>=width:
        raise AttributeError('Error')
    flow=np.zeros(shape=(height,width,2),dtype=np.float32)
    
    #np.tile用于重复数组内的数字，np.expand_dims用于修改数组维度
    grid_x=np.tile(np.expand_dims(np.arange(width),0),[height,1])
    grid_y=np.tile(np.expand_dims(np.arange(height),1),[1,width])
    
    grid_x0 = np.tile(np.array([x0]), [height, width])
    grid_y0 = np.tile(np.array([y0]), [height, width])
    
    flow[...,0]=grid_x0-grid_x
    flow[...,1]=grid_y0-grid_y
    
    return flow

In [27]:
center=[5,5]
flow=gen_flow_circle(center,height=11,width=11)
print(flow)

[[[ 5.  5.]
  [ 4.  5.]
  [ 3.  5.]
  [ 2.  5.]
  [ 1.  5.]
  [ 0.  5.]
  [-1.  5.]
  [-2.  5.]
  [-3.  5.]
  [-4.  5.]
  [-5.  5.]]

 [[ 5.  4.]
  [ 4.  4.]
  [ 3.  4.]
  [ 2.  4.]
  [ 1.  4.]
  [ 0.  4.]
  [-1.  4.]
  [-2.  4.]
  [-3.  4.]
  [-4.  4.]
  [-5.  4.]]

 [[ 5.  3.]
  [ 4.  3.]
  [ 3.  3.]
  [ 2.  3.]
  [ 1.  3.]
  [ 0.  3.]
  [-1.  3.]
  [-2.  3.]
  [-3.  3.]
  [-4.  3.]
  [-5.  3.]]

 [[ 5.  2.]
  [ 4.  2.]
  [ 3.  2.]
  [ 2.  2.]
  [ 1.  2.]
  [ 0.  2.]
  [-1.  2.]
  [-2.  2.]
  [-3.  2.]
  [-4.  2.]
  [-5.  2.]]

 [[ 5.  1.]
  [ 4.  1.]
  [ 3.  1.]
  [ 2.  1.]
  [ 1.  1.]
  [ 0.  1.]
  [-1.  1.]
  [-2.  1.]
  [-3.  1.]
  [-4.  1.]
  [-5.  1.]]

 [[ 5.  0.]
  [ 4.  0.]
  [ 3.  0.]
  [ 2.  0.]
  [ 1.  0.]
  [ 0.  0.]
  [-1.  0.]
  [-2.  0.]
  [-3.  0.]
  [-4.  0.]
  [-5.  0.]]

 [[ 5. -1.]
  [ 4. -1.]
  [ 3. -1.]
  [ 2. -1.]
  [ 1. -1.]
  [ 0. -1.]
  [-1. -1.]
  [-2. -1.]
  [-3. -1.]
  [-4. -1.]
  [-5. -1.]]

 [[ 5. -2.]
  [ 4. -2.]
  [ 3. -2.]
  [ 2. -2.]
  [ 1. -2.]
  

将光流数据可视化
将光流数据可视化的方式有很多，具体可以参考https://blog.csdn.net/tywwwww/article/details/126125681
下面用torchversion来进行可视化

In [28]:
import torch
import torchvision
from torchvision.utils import flow_to_image

def flow_to_image_torch(flow):
    """
    
    :param flow: 接收光流矩阵，矩阵的输入类型为np.array
    :return: 光流的可视化结果，输出类型为np.array
    """
    #np.transpose(flow,[2,0,1])将原来flow[高,宽,通道数]转化为[通道数,高,宽]
    #再利用torch.from_numpy转化为tensor
    flow=torch.from_numpy(np.transpose(flow,[2,0,1]))
    flow_im=flow_to_image(flow)
    
    #将flow.im转化为numpy再将通道数转回去
    img=np.transpose(flow_im.numpy(),[1,2,0])
    print(img.shape)
    return img

In [30]:
flow_img=flow_to_image_torch(flow)
print(type(flow_img))

(11, 11, 3)
<class 'numpy.ndarray'>


将numpy转化为图片输出

In [33]:
from PIL import Image
flow_img_show=Image.fromarray(flow_img)
flow_img_show.show()
#flow_img_show.save('result.jpg')

光流映射(Warp):
将光流应用到某一张图像中

In [ ]:
#方式一：
import numpy as np

def image_warp(im,flow,mode='bilinear'):
    """
    该函数可以对输入图像按照给定光流变换
    :param im: 输入图片，图片数据类型为np.array
    :param flow: 光流，光流的数据类型为np.array
    :param mode: 对光流插值的方式，使得光流可以与图像大小匹配
    :return:转换后的图片，图片类型为np.array
    """
    flag = 4
    if im.ndim == 2:
        height, width = im.shape
        num_batch = 1
        channels = 1
        im = im[np.newaxis, :, :, np.newaxis]
        flow = flow[np.newaxis, :, :]
        flag = 2
    elif im.ndim == 3:
        height, width, channels = im.shape
        num_batch = 1
        im = im[np.newaxis, :, :]
        flow = flow[np.newaxis, :, :]
        flag = 3
    elif im.ndim == 4:
        num_batch, height, width, channels = im.shape
        flag = 4
    else:
        raise AttributeError('The dimension of im must be 2, 3 or 4')

    max_x = width - 1
    max_y = height - 1
    zero = 0

    # We have to flatten our tensors to vectorize the interpolation
    im_flat = np.reshape(im, [-1, channels])
    flow_flat = np.reshape(flow, [-1, 2])

    # Floor the flow, as the final indices are integers
    flow_floor = np.floor(flow_flat).astype(np.int32)

    # Construct base indices which are displaced with the flow
    pos_x = np.tile(np.arange(width), [height * num_batch])
    grid_y = np.tile(np.expand_dims(np.arange(height), 1), [1, width])
    pos_y = np.tile(np.reshape(grid_y, [-1]), [num_batch])

    x = flow_floor[:, 0]
    y = flow_floor[:, 1]

    x0 = pos_x + x
    y0 = pos_y + y

    x0 = np.clip(x0, zero, max_x)
    y0 = np.clip(y0, zero, max_y)

    dim1 = width * height
    batch_offsets = np.arange(num_batch) * dim1
    base_grid = np.tile(np.expand_dims(batch_offsets, 1), [1, dim1])
    base = np.reshape(base_grid, [-1])

    base_y0 = base + y0 * width

    if mode == 'nearest':
        idx_a = base_y0 + x0
        warped_flat = im_flat[idx_a]
    elif mode == 'bilinear':
        # The fractional part is used to control the bilinear interpolation.
        bilinear_weights = flow_flat - np.floor(flow_flat)

        xw = bilinear_weights[:, 0]
        yw = bilinear_weights[:, 1]

        # Compute interpolation weights for 4 adjacent pixels
        # expand to num_batch * height * width x 1 for broadcasting in add_n below
        wa = np.expand_dims((1 - xw) * (1 - yw), 1) # top left pixel
        wb = np.expand_dims((1 - xw) * yw, 1) # bottom left pixel
        wc = np.expand_dims(xw * (1 - yw), 1) # top right pixel
        wd = np.expand_dims(xw * yw, 1) # bottom right pixel

        x1 = x0 + 1
        y1 = y0 + 1

        x1 = np.clip(x1, zero, max_x)
        y1 = np.clip(y1, zero, max_y)

        base_y1 = base + y1 * width
        idx_a = base_y0 + x0
        idx_b = base_y1 + x0
        idx_c = base_y0 + x1
        idx_d = base_y1 + x1

        Ia = im_flat[idx_a]
        Ib = im_flat[idx_b]
        Ic = im_flat[idx_c]
        Id = im_flat[idx_d]

        warped_flat = wa * Ia + wb * Ib + wc * Ic + wd * Id
    warped = np.reshape(warped_flat, [num_batch, height, width, channels])

    if flag == 2:
        warped = np.squeeze(warped)
    elif flag == 3:
        warped = np.squeeze(warped, axis=0)
    else:
        pass
    warped = warped.astype(np.uint8)

    return warped

In [ ]:
#方式2(待补充)：
#使用torch.nn.functional.grid_sample函数采样(有些论文中是这样使用了这样的方式)
#详细一点的解释：https://blog.csdn.net/qq_40968179/article/details/128093033

参考：https://blog.csdn.net/qq_33757398/article/details/106332814?spm=1001.2014.3001.5506